# wrappers

> Fill in a module description here

In [ ]:
#| default_exp wrappers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# Goal: tool wrapper for GraphManager.query
from cogitarelink.core.graph import GraphManager
from dspy import Module, Signature

class QueryInstances(Module):
    """QueryInstances(subj:str='', pred:str='', obj:str='') ➜ list[tuple]
    Layer: Instances. Perform triple‑pattern query on the active GraphManager.
    """
    signature = Signature("subj:str pred:str obj:str -> triples:list")

    def __init__(self, graph:GraphManager):
        super().__init__()
        self.graph = graph

    def forward(self, subj:str="", pred:str="", obj:str=""):
        return self.graph.query(subj=subj or None, pred=pred or None, obj=obj or None)

# quick smoke‑test
g = GraphManager(use_rdflib=True)
g.add_triple("A","knows","B")
assert QueryInstances(g)().startswith([("A","knows","B")])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)

In [ ]:
from inspect import signature
import dspy, importlib

def make_tools(registry):
    tools = []
    for name, meta in registry.items():
        module = importlib.import_module(f"cogitarelink.{meta.get('submod','core')}")
        target = getattr(module, name)
        sig_str = " ".join(f"{k}:{p.annotation.__name__}"
                           for k,p in signature(target).parameters.items())
        sig_str += " -> output"

        class _Tool(dspy.Module):
            """{meta['doc']}  [Layer: {meta['layer']}]"""
            signature = dspy.Signature(sig_str)
            def forward(self, **kw): return target(**kw)

        _Tool.__name__ = meta["tool"]
        tools.append(_Tool)
    return tools

/Users/cvardema/dev/git/LA3D/cogitarelink/cogitarelink-dspy/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()